In [1]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import geopandas as gp

In [2]:
from shapely.geometry import Point

In [3]:
surplus = pd.read_csv('data/surplus.csv')
surplus.head()
centroids = pd.DataFrame([])
centroids['county'] = surplus['County Name']
centroids['lon'] = surplus.CENTROID.apply(lambda x: x[7:22])
centroids['lat'] = surplus.CENTROID.apply(lambda x:x[26:42])
distance_matrix = pd.DataFrame([],columns=['county_A','county_B','distance'])

In [6]:
prueba = gp.geodataframe. .f('data/surplus.csv')

AttributeError: 'module' object has no attribute 'fromfile'

In [ ]:
for rows in centroids.index:
    try:
        centroids.iloc[rows].lon = float(centroids.iloc[rows].lon)
    except ValueError:
        print(centroids.iloc[rows].lon)

In [37]:
df = pd.DataFrame(surplus.CENTROID.str.split(' ',2).tolist(),
                                   columns = ['point','flips','row'])
df['lat'] = df['flips'].apply(lambda x:x[1:])

In [38]:
df

,point,flips,row,lat
0,POINT,(-86.64274644826507,32.53492681419619),-86.64274644826507
1,POINT,(-87.72256254273498,30.72757424278549),-87.72256254273498
2,POINT,(-85.39322863754059,31.86956145298493),-85.39322863754059
3,POINT,(-87.12646629573119,32.9986320030611),-87.12646629573119
4,POINT,(-86.5673817538303,33.98086991880072),-86.5673817538303
5,POINT,(-85.71567917787191,32.10053526617886),-85.71567917787191
6,POINT,(-86.68030021648595,31.75244652199047),-86.68030021648595
7,POINT,(-85.82601616346315,33.77143675523726),-85.82601616346315
8,POINT,(-85.39238174532437,32.91447723862047),-85.39238174532437
9,POINT,(-85.60379505960289,34.17592267370561),-85.60379505960289


In [2]:
def distance_geog(lat1,lon1,lat2,lon2):
    # http://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude-python
    # approximate radius of earth in miles
    R = 3959.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [ ]:
for county_A in surplus:
    for county_B in surplus:
        distance_matrix['county_A'] = county_A['name']
        distance_matrix['county_B'] = county_B['name']
        distance_matrix['distance'] = distance_geog(county_A['lat'],county_A['lon'],county_B['lat'],county_B['lon'])